In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
dvfs_list = ['1', '0c00', '0d00', '0e00', 
             '0f00', '1000', '1100', '1200', 
             '1300', '1400', '1500', '1600', '1700', '1800', '1900', '1a00']
dvfs_dict = {
    "0x0c00" :  1.2,
    "0x0d00" :  1.3,
    "0x0e00" :  1.4,
    "0x0f00" :  1.5,
    "0x1000" : 1.6,
    "0x1100" : 1.7,
    "0x1200" : 1.8,
    "0x1300" : 1.9,
    "0x1400" : 2.0,
    "0x1500" : 2.1,
    "0x1600" : 2.2,
    "0x1700" : 2.3,
    "0x1800" : 2.4,
    "0x1900" : 2.5,
    "0x1a00" : 2.6,
    "0x1b00" : 2.7,
    "0x1c00" : 2.8,
    "0x1d00" : 2.9,
}

df_dict = {
    'i': [], 'itr': [], 'dvfs': [], 'rate': [], 'policy': [], 'watts_avg': [], 'watts_std': [], 
    
    'pollCnt': [], 'c1Cnt': [], 'c1eCnt': [],'c3Cnt': [], 'c6Cnt': [], 
    'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [],
    
    'SinknumRecordsInPerSecond_avg': [], 'SinknumRecordsInPerSecond_std': [], 
    'SinknumRecordsOutPerSecond_avg': [], 'SinknumRecordsOutPerSecond_std': [], 
    'SinkbusyTimeMsPerSecond_avg': [], 'SinkbusyTimeMsPerSecond_std': [], 
    'SinkbackPressuredTimeMsPerSecond_avg': [], 'SinkbackPressuredTimeMsPerSecond_std': [], 
    'SinkbusyTime_%': [], 'SinkbackPressuredTime_%': [], 

    'SourcenumRecordsInPerSecond_avg': [], 'SourcenumRecordsInPerSecond_std': [], 
    'SourcenumRecordsOutPerSecond_avg': [], 'SourcenumRecordsOutPerSecond_std': [], 
    'SourcebusyTimeMsPerSecond_avg': [], 'SourcebusyTimeMsPerSecond_std': [], 
    'SourcebackPressuredTimeMsPerSecond_avg': [], 'SourcebackPressuredTimeMsPerSecond_std': [], 
    'SourcebusyTime_%': [], 'SourcebackPressuredTime_%': [], 

    'MappernumRecordsInPerSecond_avg': [], 
    'MappernumRecordsInPerSecond_std': [], 'MappernumRecordsOutPerSecond_avg': [], 
    'MappernumRecordsOutPerSecond_std': [], 'MapperbusyTimeMsPerSecond_avg': [], 
    'MapperbusyTimeMsPerSecond_std': [], 'MapperbackPressuredTimeMsPerSecond_avg': [], 
    'MapperbackPressuredTimeMsPerSecond_std': [],
    'MapperbusyTime_%': [], 'MapperbackPressuredTime_%': []
}

print(df_dict)
print("*****************************************************************")

def parseFile(rate, itr, dvfs, policy, i):
    file=f"/home/han/github/logs/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_repeat{i}/summary.csv"
    print(file)
    df_dict['i'].append(i)
    df_dict['itr'].append(itr)
    if '0x'+dvfs in dvfs_dict:
        df_dict['dvfs'].append(dvfs_dict['0x'+dvfs])
    else:
        df_dict['dvfs'].append(dvfs)
    df_dict['policy'].append(policy)
    df_dict['rate'].append(rate)
        
    df = pd.read_csv(file)
    df = df[df.columns.drop(list(df.filter(regex='Cnt')))]
    df = df[df.columns.drop(list(df.filter(regex='Bytes')))]
    #print(df)
    
    dff = df[df['name'].str.contains('Sink')]
    dff.columns = 'Sink' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    dff = df[df['name'].str.contains('Source')]
    dff.columns = 'Source' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    dff = df[df['name'].str.contains('Mapper')]
    dff.columns = 'Mapper' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    jfile = f"/home/han/github/logs/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_repeat{i}/server2_rapl.log"
    with open(jfile) as file:
        lines = [float(line.rstrip()) for line in file]
        print(len(lines))
        df_dict['watts_avg'].append(float(round(np.mean(lines[300:500]), 2)))
        df_dict['watts_std'].append(float(round(np.std(lines[300:500]), 2)))
        #print(np.std(lines[300:500]))
        
    jfile = f"/home/han/github/logs/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_repeat{i}/stats.csv"
    with open(jfile) as file:
        #$POLL, $C1, $C1E, $C3, $C6, $RXP, $RXB, $TXP, $TXB
        # 'pollCnt': [], 'c1Cnt': [], 'c1eCnt': [],'c3Cnt': [], 'c6Cnt': [], 'rxBytes': [], 'txBytes': [],
        poll = []
        c1 = []
        c1e = []
        c3 = []
        c6 = []
        rxp = []
        rxb = []
        txp = []
        txb = []
        for line in file:
            ll = [int(a) for a in line.strip().split(',')]
            poll.append(ll[0])
            c1.append(ll[1])
            c1e.append(ll[2])
            c3.append(ll[3])
            c6.append(ll[4])
            rxp.append(ll[5])
            rxb.append(ll[6])
            txp.append(ll[7])
            txb.append(ll[8])
        df_dict['pollCnt'].append(np.sum(poll[30:50]))
        df_dict['c1Cnt'].append(np.sum(c1[30:50]))
        df_dict['c1eCnt'].append(np.sum(c1e[30:50]))
        df_dict['c3Cnt'].append(np.sum(c3[30:50]))
        df_dict['c6Cnt'].append(np.sum(c6[30:50]))
        #'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [],
        df_dict['rxPackets'].append(np.sum(rxp[30:50]))
        df_dict['rxBytes'].append(np.sum(rxb[30:50]))
        df_dict['txPackets'].append(np.sum(txp[30:50]))
        df_dict['txBytes'].append(np.sum(txb[30:50]))
  

{'i': [], 'itr': [], 'dvfs': [], 'rate': [], 'policy': [], 'watts_avg': [], 'watts_std': [], 'pollCnt': [], 'c1Cnt': [], 'c1eCnt': [], 'c3Cnt': [], 'c6Cnt': [], 'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [], 'SinknumRecordsInPerSecond_avg': [], 'SinknumRecordsInPerSecond_std': [], 'SinknumRecordsOutPerSecond_avg': [], 'SinknumRecordsOutPerSecond_std': [], 'SinkbusyTimeMsPerSecond_avg': [], 'SinkbusyTimeMsPerSecond_std': [], 'SinkbackPressuredTimeMsPerSecond_avg': [], 'SinkbackPressuredTimeMsPerSecond_std': [], 'SinkbusyTime_%': [], 'SinkbackPressuredTime_%': [], 'SourcenumRecordsInPerSecond_avg': [], 'SourcenumRecordsInPerSecond_std': [], 'SourcenumRecordsOutPerSecond_avg': [], 'SourcenumRecordsOutPerSecond_std': [], 'SourcebusyTimeMsPerSecond_avg': [], 'SourcebusyTimeMsPerSecond_std': [], 'SourcebackPressuredTimeMsPerSecond_avg': [], 'SourcebackPressuredTimeMsPerSecond_std': [], 'SourcebusyTime_%': [], 'SourcebackPressuredTime_%': [], 'MappernumRecordsInPerSecond_avg'

In [3]:
itr=1
dvfs="1"
nrepeat = 2

for i in range(nrepeat):
    for rate in [100000, 200000, 300000]:
        for policy in ["powersave", "performance", "ondemand", "conservative", "schedutil"]:
            parseFile(rate, itr, dvfs, policy, i)
print(df_dict)

/home/han/github/logs/query1_cores16_frate100000_600000_fbuff-1_itr1_powersavedvfs1_repeat0/summary.csv
719
/home/han/github/logs/query1_cores16_frate100000_600000_fbuff-1_itr1_performancedvfs1_repeat0/summary.csv
706
/home/han/github/logs/query1_cores16_frate100000_600000_fbuff-1_itr1_ondemanddvfs1_repeat0/summary.csv
711
/home/han/github/logs/query1_cores16_frate100000_600000_fbuff-1_itr1_conservativedvfs1_repeat0/summary.csv
712
/home/han/github/logs/query1_cores16_frate100000_600000_fbuff-1_itr1_schedutildvfs1_repeat0/summary.csv
708
/home/han/github/logs/query1_cores16_frate200000_600000_fbuff-1_itr1_powersavedvfs1_repeat0/summary.csv
717
/home/han/github/logs/query1_cores16_frate200000_600000_fbuff-1_itr1_performancedvfs1_repeat0/summary.csv
706
/home/han/github/logs/query1_cores16_frate200000_600000_fbuff-1_itr1_ondemanddvfs1_repeat0/summary.csv
710
/home/han/github/logs/query1_cores16_frate200000_600000_fbuff-1_itr1_conservativedvfs1_repeat0/summary.csv
718
/home/han/github/log

In [6]:
d = pd.DataFrame(df_dict)
print(d.columns)
d0 = d[(d.i == 0)]
#print(d.groupby(i))
d0[['i', 'rate', 'policy', 
   'SourcenumRecordsOutPerSecond_avg', 'MapperbusyTimeMsPerSecond_avg',
   'watts_avg', 'watts_std', 'rxBytes', 'txBytes']]

Index(['i', 'itr', 'dvfs', 'rate', 'policy', 'watts_avg', 'watts_std',
       'pollCnt', 'c1Cnt', 'c1eCnt', 'c3Cnt', 'c6Cnt', 'rxPackets', 'rxBytes',
       'txPackets', 'txBytes', 'SinknumRecordsInPerSecond_avg',
       'SinknumRecordsInPerSecond_std', 'SinknumRecordsOutPerSecond_avg',
       'SinknumRecordsOutPerSecond_std', 'SinkbusyTimeMsPerSecond_avg',
       'SinkbusyTimeMsPerSecond_std', 'SinkbackPressuredTimeMsPerSecond_avg',
       'SinkbackPressuredTimeMsPerSecond_std', 'SinkbusyTime_%',
       'SinkbackPressuredTime_%', 'SourcenumRecordsInPerSecond_avg',
       'SourcenumRecordsInPerSecond_std', 'SourcenumRecordsOutPerSecond_avg',
       'SourcenumRecordsOutPerSecond_std', 'SourcebusyTimeMsPerSecond_avg',
       'SourcebusyTimeMsPerSecond_std',
       'SourcebackPressuredTimeMsPerSecond_avg',
       'SourcebackPressuredTimeMsPerSecond_std', 'SourcebusyTime_%',
       'SourcebackPressuredTime_%', 'MappernumRecordsInPerSecond_avg',
       'MappernumRecordsInPerSecond_std', 'Ma

,i,rate,policy,SourcenumRecordsOutPerSecond_avg,MapperbusyTimeMsPerSecond_avg,watts_avg,watts_std,rxBytes,txBytes
0,0,100000,powersave,100000.000000,459.331250,45.19,1.12,29612586601,11990886019
1,0,100000,performance,99982.635159,192.104167,47.92,1.84,29194546985,11835025938
2,0,100000,ondemand,100000.000000,222.664583,47.36,2.06,29237561807,11829796037
3,0,100000,conservative,99990.579524,221.612500,45.87,2.07,29382161580,11890895229
4,0,100000,schedutil,100000.000000,230.893750,45.13,2.23,29185817247,11808743707
5,0,200000,powersave,199980.334325,862.362500,62.59,0.39,59776528897,23978155826
6,0,200000,performance,200000.000000,404.791667,70.40,1.60,58773537527,23615684821
7,0,200000,ondemand,200000.000000,397.093750,68.55,1.66,59002482603,23666341976
8,0,200000,conservative,200000.000000,405.512500,65.76,1.61,59974079098,24041260707
9,0,200000,schedutil,200000.000000,400.335417,67.47,1.63,58976058227,23673560789


In [7]:
d = pd.DataFrame(df_dict)
print(d.columns)
d1 = d[(d.i == 1)]
#print(d.groupby(i))
d1[['i', 'rate', 'policy', 
   'SourcenumRecordsOutPerSecond_avg', 'MapperbusyTimeMsPerSecond_avg',
   'watts_avg', 'watts_std', 'rxBytes', 'txBytes']]

Index(['i', 'itr', 'dvfs', 'rate', 'policy', 'watts_avg', 'watts_std',
       'pollCnt', 'c1Cnt', 'c1eCnt', 'c3Cnt', 'c6Cnt', 'rxPackets', 'rxBytes',
       'txPackets', 'txBytes', 'SinknumRecordsInPerSecond_avg',
       'SinknumRecordsInPerSecond_std', 'SinknumRecordsOutPerSecond_avg',
       'SinknumRecordsOutPerSecond_std', 'SinkbusyTimeMsPerSecond_avg',
       'SinkbusyTimeMsPerSecond_std', 'SinkbackPressuredTimeMsPerSecond_avg',
       'SinkbackPressuredTimeMsPerSecond_std', 'SinkbusyTime_%',
       'SinkbackPressuredTime_%', 'SourcenumRecordsInPerSecond_avg',
       'SourcenumRecordsInPerSecond_std', 'SourcenumRecordsOutPerSecond_avg',
       'SourcenumRecordsOutPerSecond_std', 'SourcebusyTimeMsPerSecond_avg',
       'SourcebusyTimeMsPerSecond_std',
       'SourcebackPressuredTimeMsPerSecond_avg',
       'SourcebackPressuredTimeMsPerSecond_std', 'SourcebusyTime_%',
       'SourcebackPressuredTime_%', 'MappernumRecordsInPerSecond_avg',
       'MappernumRecordsInPerSecond_std', 'Ma

,i,rate,policy,SourcenumRecordsOutPerSecond_avg,MapperbusyTimeMsPerSecond_avg,watts_avg,watts_std,rxBytes,txBytes
15,1,100000,powersave,99994.334881,455.545833,45.22,1.41,29555190640,11967485023
16,1,100000,performance,99998.804762,190.712500,47.34,1.85,29058370015,11776306304
17,1,100000,ondemand,99999.611587,197.052083,45.72,2.10,29333933890,11871467389
18,1,100000,conservative,99993.352778,227.577083,44.86,1.97,29263180109,11849011761
19,1,100000,schedutil,100000.000000,216.454167,44.89,2.07,29331715987,11867429490
20,1,200000,powersave,199973.677183,855.631250,62.37,0.45,59741522727,23966260985
21,1,200000,performance,200000.000000,431.841667,70.89,1.50,58467372578,23461272011
22,1,200000,ondemand,200000.000000,383.856250,68.15,1.60,58883569934,23648133761
23,1,200000,conservative,200000.000000,407.222917,66.99,1.64,58969200009,23656003830
24,1,200000,schedutil,200000.000000,416.416667,67.96,1.87,58697675985,23557912552
